In [ ]:
import pandas as pd
import numpy as np

In [ ]:
player_stats = pd.read_json("./data/player_all_stat.json")
player_stats

In [ ]:
player_stats.describe(include='all').T

In [ ]:
player_stats.dtypes

In [ ]:
player_stats.nr.duplicated().any()

In [ ]:
player_stats.sort_values('mean', ascending=False)

In [ ]:
promising_players = player_stats.query("nr >= 5 & mean >= 75 & std <= 50")
promising_players

In [ ]:
promising_players.sort_values('mean', ascending=False)

In [ ]:
anonymous_player = promising_players.loc[0]
anonymous_player

In [ ]:
promising_players = promising_players.drop([0], axis=0)
promising_players

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled = scaler.fit_transform(promising_players)
scaled = pd.DataFrame(scaled, columns=promising_players.columns, index=promising_players.index)
scaled

In [ ]:
scaled['mean'].isna().any()

In [ ]:
np.sqrt(scaled['nr']).isna().any()

In [ ]:
(1 / (scaled['std'] + 1)).isna().any()

In [ ]:
scores = (scaled['mean'] * (1/(scaled['std']+1)) * np.sqrt(scaled['nr']))
scores.isna().any()

In [ ]:
scores = scores / scores.max()

In [ ]:
scores

In [ ]:
scores[scores.isna()]  # no nan

In [ ]:
scores.shape

In [ ]:
# higher score = better
# lower std = better
# more plays = better

promising_players['score'] = scores
promising_players

In [ ]:
promising_players['score'].isna().any()

In [ ]:
anonymous_player_df = pd.DataFrame(anonymous_player).T
anonymous_player_df['id'] = anonymous_player_df['id'].astype(int)
anonymous_player_df['nr'] = anonymous_player_df['nr'].astype(int)
anonymous_player_df['score'] = 1
anonymous_player_df

In [ ]:
promising_players = pd.concat([anonymous_player_df, promising_players])
promising_players

In [ ]:
promising_players = promising_players.sort_values('score', ascending=False)
promising_players

In [ ]:
promising_players.to_csv("./data/promising_players.csv")